In [1]:
import os
import time 
import datetime

def print_table(data: dict[str, list[float]], precision: int = 4):
    # Convert to list of keys and find number of rows
    headers = list(data.keys())
    headers = ["index"] + headers        
    n_rows = len(next(iter(data.values())))
    data["index"] = list(range(1, n_rows+1))

    # Format floats to desired precision
    fmt = f"{{:.{precision}f}}"

    # Compute column widths
    col_widths = []
    for h in headers:
        values = data[h]
        max_val_len = max(len(fmt.format(v)) for v in values)
        col_widths.append(max(len(h), max_val_len))

    # Build header row
    header_row = " | ".join(f"{h:>{w}}" for h, w in zip(headers, col_widths))
    separator = "-+-".join("-" * w for w in col_widths)

    # Print table
    print(header_row)
    print(separator)
    for i in range(n_rows):
        row = " | ".join(fmt.format(data[h][i]).rjust(w) for h, w in zip(headers, col_widths))
        print(row)

In [2]:
def dt_to_intkey(date_str: str, time_str: str) -> float:
    dt = datetime.datetime.strptime(f"{date_str} {time_str}", "%Y_%m_%d %H_%M_%S")
    return dt.timestamp()

In [8]:
data_dir = "../../logs/logs/results"
res_files_unfiltered = [f for f in os.listdir(data_dir) if f.endswith(".log")]
res_files_unfiltered.sort()
# print(*res_files, sep="\n")
dates, times = zip(*(s.split("__")[:2] for s in res_files_unfiltered))
creation_times = {
    # key is date_time string converted to int/float for sorting
    index: dt_to_intkey(date, time)
    for index, (date, time) in enumerate(zip(dates, times))
}
only_after = dt_to_intkey("2025_10_30", "00_00_00")
res_files = [
    res_files_unfiltered[index]
    for index, creation_time in creation_times.items()
    if creation_time >= only_after
]
# print(*res_files, sep="\n")
print(len(res_files), "files found after filtering.")

95 files found after filtering.


In [ ]:
# for f in res_files:
#     with open(os.path.join(data_dir, f), "r") as infile:
#         lines = infile.readlines()
    
#     if not any("Final results after quantization" in line for line in lines):
#         print(f"Skipping {f} as it does not contain final results.")
#         # delete file
#         os.remove(os.path.join(data_dir, f))
#         continue

Skipping 2025_10_30__15_56_24__coolchic_kodak_kodim09.log as it does not contain final results.
Skipping 2025_10_30__15_56_24__coolchic_kodak_kodim10.log as it does not contain final results.
Skipping 2025_10_30__15_56_24__coolchic_kodak_kodim11.log as it does not contain final results.
Skipping 2025_10_30__15_56_24__coolchic_kodak_kodim12.log as it does not contain final results.
Skipping 2025_10_30__15_56_24__coolchic_kodak_kodim13.log as it does not contain final results.
Skipping 2025_10_30__15_56_24__coolchic_kodak_kodim14.log as it does not contain final results.
Skipping 2025_10_30__15_56_24__coolchic_kodak_kodim15.log as it does not contain final results.
Skipping 2025_10_30__15_56_24__coolchic_kodak_kodim16.log as it does not contain final results.
Skipping 2025_10_30__15_56_31__coolchic_kodak_kodim24.log as it does not contain final results.
Skipping 2025_10_30__15_56_32__coolchic_kodak_kodim21.log as it does not contain final results.
Skipping 2025_10_30__15_56_32__coolchic_

In [34]:
results_y_co_cg = [[{}] for _ in range(24)]
results_rgb = [[{}] for _ in range(24)]

for f in res_files:
    with open(os.path.join(data_dir, f), "r") as infile:
        lines = infile.readlines()
        ycocg = True
        for line in lines:
            if "RGB" in line:
                ycocg = False
            if "Final results after quantization" in line:
                parts = (
                    line[len("Final results after quantization:") :]
                    .strip()
                    .split(", ")
                )
                im_index = (
                    int(f.split("_")[-1][len("kodim") :][: -len(".log")]) - 1
                )
                for part in parts:
                    key, value = part.split(": ")
                    if ycocg:
                        results_y_co_cg[im_index][-1][key] = float(value)
                    else:
                        results_rgb[im_index][-1][key] = float(value)

                if ycocg and len(results_y_co_cg[im_index]) < 2:
                    results_y_co_cg[im_index].append({})
                elif not ycocg and len(results_rgb[im_index]) < 2:
                    results_rgb[im_index].append({})


def list_dict_to_dict_list(
    lst: list[dict[str, float]],
) -> dict[str, list[float]]:
    if not lst:
        return {}
    keys = lst[0].keys()
    dict_list = {key: [] for key in keys}
    for d in lst:
        for key in keys:
            dict_list[key].append(d.get(key, 0.0))
    return dict_list


for i, lst in enumerate(results_y_co_cg):
    if len(lst[1].keys()) > 0 and lst[0]["Loss"] > lst[1]["Loss"]:
        results_y_co_cg[i][0], results_y_co_cg[i][1] = (
            results_y_co_cg[i][1],
            results_y_co_cg[i][0],
        )

for i, lst in enumerate(results_rgb):
    if len(lst[1].keys()) > 0 and lst[0]["Loss"] > lst[1]["Loss"]:
        results_rgb[i][0], results_rgb[i][1] = (
            results_rgb[i][1],
            results_rgb[i][0],
        )

results_y_co_cg_with_arm = [d[0] for d in results_y_co_cg]
results_y_co_cg_without_arm = [d[1] for d in results_y_co_cg]
results_y_co_cg_without_arm.remove({})
results_rgb_with_arm = [d[0] for d in results_rgb]
results_rgb_without_arm = [d[1] for d in results_rgb]

results_y_co_cg_with_arm = list_dict_to_dict_list(results_y_co_cg_with_arm)
results_y_co_cg_without_arm = list_dict_to_dict_list(results_y_co_cg_without_arm)
results_rgb_with_arm = list_dict_to_dict_list(results_rgb_with_arm)
results_rgb_without_arm = list_dict_to_dict_list(results_rgb_without_arm)

print("YCoCg_with_ARM Results:")
print_table(results_y_co_cg_with_arm)
for key, values in results_y_co_cg_with_arm.items():
    avg = sum(values) / len(values) if values else 0
    print(f"{key}: {avg:.4f}")

print("YCoCg_without_ARM Results:")
print_table(results_y_co_cg_without_arm)
for key, values in results_y_co_cg_without_arm.items():
    avg = sum(values) / len(values) if values else 0
    print(f"{key}: {avg:.4f}")

print("RGB_with_ARM Results:")
print_table(results_rgb_with_arm)
for key, values in results_rgb_with_arm.items():
    avg = sum(values) / len(values) if values else 0
    print(f"{key}: {avg:.4f}")
print("RGB_without_ARM Results:")
print_table(results_rgb_without_arm)
for key, values in results_rgb_without_arm.items():
    avg = sum(values) / len(values) if values else 0
    print(f"{key}: {avg:.4f}")



YCoCg_with_ARM Results:
  index |   Loss | Rate NN | Rate Latent | Rate Img
--------+--------+---------+-------------+---------
 1.0000 | 2.4758 |  0.0108 |      0.0995 |   2.3655
 2.0000 | 2.2201 |  0.0040 |      0.0107 |   2.2054
 3.0000 | 1.7118 |  0.0083 |      0.0226 |   1.6809
 4.0000 | 2.0756 |  0.0089 |      0.0352 |   2.0316
 5.0000 | 2.4602 |  0.0075 |      0.0339 |   2.4188
 6.0000 | 2.1415 |  0.0080 |      0.0096 |   2.1239
 7.0000 | 2.3038 |  0.0028 |      0.0073 |   2.2937
 8.0000 | 2.5193 |  0.0079 |      0.0499 |   2.4616
 9.0000 | 2.2469 |  0.0083 |      0.0276 |   2.2110
10.0000 | 2.0258 |  0.0084 |      0.0045 |   2.0128
11.0000 | 2.0846 |  0.0072 |      0.0065 |   2.0709
12.0000 | 2.1113 |  0.0074 |      0.0202 |   2.0837
13.0000 | 2.9442 |  0.0071 |      0.0402 |   2.8968
14.0000 | 2.5671 |  0.0091 |      0.0403 |   2.5177
15.0000 | 2.5814 |  0.0028 |      0.0079 |   2.5707
16.0000 | 2.0853 |  0.0072 |      0.0143 |   2.0637
17.0000 | 1.9943 |  0.0083 |      0.0227